### Classification of Acute Lymphocytic Leukemia (ALL) with Convolutional Neural Network

Importing all the libraries

In [7]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import imageio
import numpy as np 
from timeit import default_timer as timer
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.layers import BatchNormalization

Getting the file paths for the data. The folder "ALL" has the data for Acute Lymphoblastic Leukemia (ALL) and the folder "hem" has data for the Normal. Currently we are loading only training and validation.

In [8]:

fold0_all = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0/all"
fold1_all = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_1/all"
fold2_all = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_2/all"

#hem is the normal one
fold0_nrml = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0/hem"
fold1_nrml = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_1/hem"
fold2_nrml = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_2/hem"

test_data = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data/"
test_labels = pd.read_csv("/kaggle/input/leukemia-classification/C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data_labels.csv")

In [ ]:
can_image_fnames = os.listdir(fold0_all)

##Plotting a single image
cancer_img = imageio.imread(os.path.join(fold0_all,
                                         can_image_fnames[5]))
plt.imshow(cancer_img)
plt.title('Cancer - ALL Cell')
plt.show()


In [ ]:
### Normal Cell

In [ ]:
#blood cell without cancer - normal cells
nrml_image_fnames = os.listdir(fold0_nrml)
nrml_img = imageio.imread(os.path.join(fold0_nrml,
                                         nrml_image_fnames[5]))

plt.imshow(nrml_img)
plt.title('Normal - Hem Cell')
plt.show()



The image is a 450 * 450 pixels with colors 'RGB'

In [ ]:
nrml_img.shape

#### Read all the normal and cancer images as a list. Create a dataframe with file path as one column and the cancer type as another columns

In [ ]:
def get_path_image(folder):
    image_paths = []
    image_fnames = os.listdir(folder) 
    for img_id in range(len(image_fnames)):
        img = os.path.join(folder,image_fnames[img_id])
        image_paths.append(img)
    
    return image_paths

The number of cancer images in the train dataset is:

In [ ]:
#image absolute paths for cancer cells and normal cells
cancer_lst = []

for i in [fold0_all,fold1_all,fold2_all]:
    paths = get_path_image(i)
    cancer_lst.extend(paths)
    
    
print(len(cancer_lst))

The number of normal images in the train dataset.

In [ ]:
normal_lst = []
for i in [fold0_nrml,fold1_nrml,fold2_nrml]:
    paths = get_path_image(i)
    normal_lst.extend(paths)
   
   
print(len(normal_lst))

In [ ]:
cancer_dict = {"x_col":cancer_lst,
          "y_col":[np.nan for x in range(len(cancer_lst))]}


cancer_dict["y_col"] = "ALL"

normal_dict = {"x_col":normal_lst,
          "y_col":[np.nan for x in range(len(normal_lst))]}


normal_dict["y_col"] = "HEM"

cancer_df = pd.DataFrame(cancer_dict)
normal_df = pd.DataFrame(normal_dict)

train_df = cancer_df.append(normal_df, ignore_index=True)

###  Bar chart for showing the number of images for each cell type.

In [ ]:

plt.bar(['Normal', 'ALL'], [len(normal_lst), len(cancer_lst)])
plt.title('Bar chart for showing the percentage of images in each cell type')
plt.show()

### Pie chart showing the percentage of each cell type

In [ ]:
plt.pie([len(train_df[train_df["y_col"]=="ALL"]),len(train_df[train_df["y_col"]=="HEM"])],
        labels=["ALL","Normal"],autopct='%.f'
       )
plt.title('Pie Chart for percentage of each cell type')
plt.gca()
plt.show()



In [ ]:
select_normal = np.random.choice(normal_lst, 3, replace = False)
select_all = np.random.choice(cancer_lst, 3, replace = False)


fig = plt.figure(figsize = (8,6))

for i in range(6):
    if i < 3:
        fp = select_normal[i]
        label = 'Normal'
    else:
        fp = select_all[i-3]
        label = 'ALL'
    ax = fig.add_subplot(2, 3, i+1)
    fn = image.load_img(fp, target_size = (100,100),
                        color_mode='rgb')
    plt.imshow(fn, cmap='Greys_r')
    plt.title(label)
    plt.axis('off')
plt.show()

Normal cells are mostly sperical and non-clefted cells with homogeneous chromatin and few vacuoles. ALL cells are irregularly shaped, clefted cells with hetrogeneous chromatin and multiple nucleoli and vacuoles.

#### Average Images
Let us average 50 cells of each sub class and see how much variation is there to identify meaningful differences.

In [ ]:
#function for processing images to numpy array for creating mean
def img2np(fn_list, size = (32, 32)):
   
    i = 0
    for fp in fn_list:
        
        current_image = image.load_img(fp, 
                                       target_size = size, 
                                       color_mode = 'grayscale')
        
        img_ts = image.img_to_array(current_image)
        img_ts = [img_ts.ravel()]
             
        
        
        
        if i == 0:
            full_mat = img_ts
            
        else: 
            full_mat = np.concatenate((full_mat, img_ts))    
        i = i + 1    
    return full_mat



normal_npArray = img2np(normal_lst)
cancer_npArray = img2np(cancer_lst)


##Let us select 50 images to make the comparison clear


nrml_images_sub = normal_npArray[:50]
cnr_images_sub = cancer_npArray[:50]


def plt_mean(mat , title, size = (32, 32)):
    
    mean_img = np.mean(mat , axis = 0)
    mean_img = mean_img.reshape(size)
    plt.imshow(mean_img, vmin=0, vmax=255)
    plt.title(f'Average {title}')
    plt.axis('off')
    plt.show()
    return mean_img

nrml_mean = plt_mean(nrml_images_sub , "Normal")
cnr_mean = plt_mean(cnr_images_sub, "Cancer - ALL")

The mean images can't detect much diferences between the classes. Though the ALL cells are bit larger than normal cell.

Reading the test and validation datset file locations and converting them to a dataframe

In [ ]:
all_len = len(cancer_df)
norm_len = len(normal_df)

nrm_size = int((norm_len/100) * 25)
all_size = int((all_len/100) * 25)
validation_df = pd.DataFrame()
validation_df = validation_df.append(normal_df[0:nrm_size])
validation_df = validation_df.append(cancer_df[0:all_size])    

train_df = pd.DataFrame()
train_df = train_df.append(normal_df[nrm_size : norm_len])
train_df = train_df.append(cancer_df[all_size : all_len])   



In [ ]:
#Reading the validation data
test_list = get_path_image(test_data)


##Convert the list to a dictionary. The labels are stored in the val_labels
#3So we create a dictionary with x as the file name and y as the labels
##The labels are having 0's and 1's.
##0 means normal and 1 means cancer - ALL

test_dict = {"x_col":test_list,
                   "y_col":test_labels["labels"]}

test_df = pd.DataFrame(test_dict)

test_df["y_col"].replace(to_replace = [1,0], value = ["ALL","HEM"], inplace = True)


### Data Preprocessing

With keras, image preprocessing has become much easier. Instead of reading subfolders for all files, load pictures and converting it to numpy arrays
Keras provides API calls. flow_from_dataframe allows us to input a pandas dataframe which contains the filenames, with or without extension, as one column and
and a column which has the class names and directly read the images from the directory with their respective class names mapped.


In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255 #pixel values are 255 maximum
         )


test_datagen = ImageDataGenerator(
        rescale=1./255 )

train_generator = train_datagen.flow_from_dataframe(
                  train_df,
                  x_col = "x_col",
                  y_col = "y_col",
                  target_size = (256, 256),
                 
                  #batch_size = 32,
                  color_mode = "rgb",
                  shuffle = True,
                  class_mode = "binary"
                  )

validation_generator = train_datagen.flow_from_dataframe(
                  validation_df,
                  x_col = "x_col",
                  y_col = "y_col",
                  target_size = (256, 256),                  
                  #batch_size = 32,
                  color_mode = "rgb",
                  shuffle = True,
                  class_mode = "binary"
                  )


test_generator = test_datagen.flow_from_dataframe(
                  test_df,
                  x_col = "x_col",
                  y_col = "y_col",
                  target_size = (256, 256),
                  color_mode = "rgb",
                  class_mode = "binary",
                  shuffle = False)





## A Model with one convolution and one dense layer
Convolutional layer uses fewer parameters by forcing input values to share the parameters.
Dense layer uses a linear operation, meaning, every output is formed by the function based on every input.
In other words, every input is forced into the function, and then the "Neural Network" learns it's relation to the output. 
There will be n*m connections, where n denotes the number of inputs and m denotes the number of outputs.

The output of the convolutional layer is formed by just a small size of inputs which depends on the filter's size and the weights are shared 
for all pixels. The output is constructed by using the same co-efficient for all the pixels by using the neighbouring pixels as inputs.

### Vanilla Model - 1C1D




In [ ]:
# CNN layer
model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation = 'relu', input_shape = (256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))

##Dense layer
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))





In [ ]:

# compiling models
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])



In [ ]:
start = timer()

history = model.fit(train_generator , 
                    epochs=5, 
                    validation_data=validation_generator, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')




#### Evaluate the 1C1D model performance:

In [ ]:
scores = model.evaluate(test_generator, verbose=1)

In [ ]:
!pip install git+https://github.com/paulgavrikov/visualkeras
import visualkeras
visualkeras.layered_view(model,legend=True,  to_file='1c1d.png')

In [ ]:
def myprint(s):
    with open('modelsummary.txt','w+') as f:
        print(s, file=f)

model.summary(print_fn=myprint)
model.summary()

### 2C1D
Add another convolutional layer so as to create a model with 2 convolution and 1 Dense.



In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_generator, 
                    epochs=5, 
                    validation_data=validation_generator, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

#### Evaluate the model with 2C1D

In [ ]:
scores = model.evaluate(test_generator, verbose=1)

### 3C1D

Create a model with 3 convolutions and one dense layer.





In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_generator, 
                    epochs=5, 
                    validation_data=validation_generator, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

##### Evaluate the model with 3 Convolutions and 1 Dense layer

In [ ]:
scores = model.evaluate(test_generator, verbose=1)

Let us now try deeper structures. Maybe a model with 2 X 2 X 1 convolution and 1 Dense layer

### 2x2x1C1D

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
               optimizer=optimizers.RMSprop(lr=1e-4),
               metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_generator, 
                    epochs=5, 
                    validation_data=validation_generator, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Evaluating the model performance.

In [ ]:
scores = model.evaluate(test_generator, verbose=1)

Try Adam optimizer insted of RMSprop for faster convergence.

### 2x2x1C1D Adam


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_generator, 
                    epochs=5, 
                    validation_data=validation_generator, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

Evaluate the prformnace of the model with Adam optimiser

In [ ]:
scores = model.evaluate(test_generator, verbose=1)

Try Batch Normalization for improving the performance 
### 2x2x1C1D BatchNorm

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(96, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
               optimizer=optimizers.RMSprop(lr=1e-4),
               metrics=['accuracy', 'Recall'])



start = timer()

history = model.fit(train_generator, 
                    epochs=5, 
                    validation_data=validation_generator, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

In [ ]:
scores = model.evaluate(test_generator, verbose=1)

Try Adam optimizer insted of RMSprop for faster convergence and also  add Batch Normalization improved performance

### 2x2x1C1D Adam/ BatchNorm



In [ ]:

model = models.Sequential()
model.add(layers.Conv2D(96, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(96, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_generator, 
                    epochs=5, 
                    validation_data=validation_generator, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

In [ ]:
scores = model.evaluate(test_generator, verbose=1)

### 2x2x1C1D Dropout

Try adding Droupout to combat overfitiing

In [9]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
               optimizer=optimizers.RMSprop(lr=1e-4),
               metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_generator, 
                    epochs=5, 
                    validation_data=validation_generator, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

In [ ]:
scores = model.evaluate(test_generator, verbose=1)

### 2X2C1D 

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_generator, 
                    epochs=8, 
                    validation_data=validation_generator, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

In [ ]:
scores = model.evaluate(test_generator, verbose=1)

### 2x2C1D Dropout

Try adding Dropout to combat overfitting.

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64, 3, activation = 'relu', input_shape=(256,256,3)))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy', 'Recall'])

start = timer()

history = model.fit(train_generator, 
                    epochs=8, 
                    validation_data=validation_generator, 
                    workers = 7
                   )

end = timer()
elapsed = end - start
print('Total Time Elapsed: ', int(elapsed//60), ' minutes ', (round(elapsed%60)), ' seconds')

In [ ]:
scores = model.evaluate(test_generator, verbose=1)

In [ ]:
model = models.Sequential()
#model.name="KAllCnn_IDB1"

model.add(tf.keras.layers.ZeroPadding2D(padding=(2, 2), input_shape= (256,256,3)))
model.add(layers.Conv2D(30, (5, 5), strides=1, padding = "valid", input_shape = (256,256,3), activation = 'relu'))
model.add(layers.Dropout(0.4))

model.add(tf.keras.layers.ZeroPadding2D(padding=(2, 2), input_shape=(256,256,3)))
model.add(layers.Conv2D(30, (5, 5), strides=1, padding = "valid", activation = 'relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2, padding = 'valid'))
model.add(layers.Dropout(0.6))

model.add(layers.Flatten())
model.add(layers.Dense(1))
model.add(layers.Activation("softmax"))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-6),
              metrics=['accuracy', 'Recall'])



history = model.fit(train_generator, 
                    epochs=8, 
                    validation_data=validation_generator, 
                    workers = 7
                   )

In [ ]:
score = model.evaluate(test_generator, verbose=1)

In [ ]:
visualkeras.layered_view(model,legend=True,  to_file='Final.png')



In [ ]:
model.summary()